In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bank-churn/sample_submission.csv
/kaggle/input/bank-churn/trainBCD.csv
/kaggle/input/bank-churn/testBCD.csv


In [2]:
# https://www.kaggle.com/competitions/playground-series-s4e1/overview #competicion
# https://www.kaggle.com/competitions/playground-series-s4e1/data # archivos
# https://www.kaggle.com/competitions/playground-series-s4e1/leaderboard # leaderboard


# CORE DATA MANIPULATION
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import math

# MACHINE LEARNING - SCIKIT-LEARN
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, RFE, RFECV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, roc_auc_score, roc_curve, auc
from sklearn.pipeline import Pipeline

# ADVANCED ML LIBRARIES
import xgboost as xgb
import lightgbm as lgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, CatBoostRegressor

# VISUALIZATION
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# CONFIGURATION FOR PLOTS
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

# UTILITIES
import os
import sys
import warnings
import itertools
from datetime import datetime, timedelta
import time
from collections import Counter
import pickle
import joblib

# JUPYTER SPECIFIC
from IPython.display import display, HTML
from tqdm.notebook import tqdm

# SUPPRESS WARNINGS
warnings.filterwarnings('ignore')

# PANDAS CONFIGURATION
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# NUMPY CONFIGURATION
np.random.seed(42)

def quick_info(df):
    """Información rápida del dataset"""
    print(f"📊 Dataset Shape: {df.shape}")
    print(f"🔢 Columnas numéricas: {df.select_dtypes(include=[np.number]).columns.tolist()}")
    print(f"📝 Columnas categóricas: {df.select_dtypes(include=['object']).columns.tolist()}")
    print(f"❌ Valores nulos por columna:")
    print(df.isnull().sum()[df.isnull().sum() > 0])

def plot_missing_values(df):
    """Visualizar valores faltantes"""
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if len(missing) > 0:
        plt.figure(figsize=(10, 6))
        missing.plot(kind='bar')
        plt.title('Valores Faltantes por Columna')
        plt.ylabel('Cantidad')
        plt.xticks(rotation=45)
        plt.show()
    else:
        print("✅ No hay valores faltantes en el dataset")



2025-07-22 20:48:31.808371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753217312.071483      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753217312.148287      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow disponible
Librerías de procesamiento de imágenes disponibles


✅ Todas las librerías importadas correctamente!
📊 Pandas version: 2.2.3
🔢 NumPy version: 1.26.4
🤖 Scikit-learn version: 1.2.2
📈 Matplotlib version: 3.7.2
🎨 Seaborn version: 0.12.2
🎯 Template listo para usar en Kaggle!


In [3]:
df = pd.read_csv("/kaggle/input/bank-churn/trainBCD.csv")
test = pd.read_csv("/kaggle/input/bank-churn/testBCD.csv")

In [4]:
df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [5]:
quick_info(df)

📊 Dataset Shape: (165034, 14)
🔢 Columnas numéricas: ['id', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']
📝 Columnas categóricas: ['Surname', 'Geography', 'Gender']
❌ Valores nulos por columna:
Series([], dtype: int64)


In [6]:
# Para empezar deberia eliminar lo que no me sirve:"Customerid","Surname"
# "id" lo guardo porque es necesario en el submission de test
# hacer one-hot encoding con "Geography","Gender"
# aplicar logaritmo a "EstimatedSalary" y a "Balance"
# esta transformacion es muy poco probable que empeore el resultado , como mucho lo mejora de forma nula o poca
# podria ver analisis de varianzas y correlaciones pero es preferible empezar con un RandomForest para saber como estamos

In [7]:
df.describe()

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,165034.0000,1.650340e+05,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000
mean,82516.5000,1.569201e+07,656.454373,38.125888,5.020353,55478.086689,1.554455,0.753954,0.497770,112574.822734,0.211599
std,47641.3565,7.139782e+04,80.103340,8.867205,2.806159,62817.663278,0.547154,0.430707,0.499997,50292.865585,0.408443
min,0.0000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,41258.2500,1.563314e+07,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74637.570000,0.000000
50%,82516.5000,1.569017e+07,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117948.000000,0.000000
75%,123774.7500,1.575682e+07,710.000000,42.000000,7.000000,119939.517500,2.000000,1.000000,1.000000,155152.467500,0.000000
max,165033.0000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [8]:
train_id = df["id"].copy()
test_id = test["id"].copy()

df1 = df.drop(["CustomerId","Surname","id"], axis=1)
df1.head()

test1 = test.drop(["CustomerId","Surname","id"], axis=1)

In [9]:
test1.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [10]:
df1["LogEstimatedSalary"] = np.log(df1["EstimatedSalary"])
test1["LogEstimatedSalary"] = np.log(test1["EstimatedSalary"])

df1 = df1.drop("EstimatedSalary", axis = 1)
test1 = test1.drop("EstimatedSalary", axis = 1)

df1.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,LogEstimatedSalary
0,668,France,Male,33.0,3,0.00,2,1.0,0.0,0,12.108735
1,627,France,Male,33.0,1,0.00,2,1.0,1.0,0,10.809799
2,678,France,Male,40.0,10,0.00,2,1.0,0.0,0,12.127390
3,581,France,Male,34.0,2,148882.54,1,1.0,1.0,0,11.345227
4,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,0,9.620384


In [11]:
df1["LogBalance"] = np.log1p(df1["Balance"])
test1["LogBalance"] = np.log1p(test1["Balance"])
df1 = df1.drop("Balance", axis = 1)
test1 = test1.drop("Balance", axis = 1)

df1.head()

,CreditScore,Geography,Gender,Age,Tenure,NumOfProducts,HasCrCard,IsActiveMember,Exited,LogEstimatedSalary,LogBalance
0,668,France,Male,33.0,3,2,1.0,0.0,0,12.108735,0.00000
1,627,France,Male,33.0,1,2,1.0,1.0,0,10.809799,0.00000
2,678,France,Male,40.0,10,2,1.0,0.0,0,12.127390,0.00000
3,581,France,Male,34.0,2,1,1.0,1.0,0,11.345227,11.91092
4,716,Spain,Male,33.0,5,2,1.0,1.0,0,9.620384,0.00000


In [12]:
df1 = pd.get_dummies(df1, columns=["Geography","Gender"])
test1 = pd.get_dummies(test1, columns=["Geography","Gender"])

df1.head()

,CreditScore,Age,Tenure,NumOfProducts,HasCrCard,IsActiveMember,Exited,LogEstimatedSalary,LogBalance,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,668,33.0,3,2,1.0,0.0,0,12.108735,0.00000,True,False,False,False,True
1,627,33.0,1,2,1.0,1.0,0,10.809799,0.00000,True,False,False,False,True
2,678,40.0,10,2,1.0,0.0,0,12.127390,0.00000,True,False,False,False,True
3,581,34.0,2,1,1.0,1.0,0,11.345227,11.91092,True,False,False,False,True
4,716,33.0,5,2,1.0,1.0,0,9.620384,0.00000,False,False,True,False,True


In [13]:
y_train = df1["Exited"]
X_train = df1.drop("Exited", axis=1)

X_test = test1


In [14]:
test1.head()

,CreditScore,Age,Tenure,NumOfProducts,HasCrCard,IsActiveMember,LogEstimatedSalary,LogBalance,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,586,23.0,2,2,0.0,1.0,11.989015,0.00000,True,False,False,True,False
1,683,46.0,2,1,1.0,0.0,11.192021,0.00000,True,False,False,True,False
2,656,34.0,7,2,1.0,0.0,11.841381,0.00000,True,False,False,True,False
3,681,36.0,8,1,1.0,0.0,11.643353,0.00000,True,False,False,False,True
4,752,38.0,10,1,1.0,0.0,11.845325,11.70573,False,True,False,False,True


In [15]:
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: Exited, dtype: int64

In [16]:
X_train.head()

,CreditScore,Age,Tenure,NumOfProducts,HasCrCard,IsActiveMember,LogEstimatedSalary,LogBalance,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,668,33.0,3,2,1.0,0.0,12.108735,0.00000,True,False,False,False,True
1,627,33.0,1,2,1.0,1.0,10.809799,0.00000,True,False,False,False,True
2,678,40.0,10,2,1.0,0.0,12.127390,0.00000,True,False,False,False,True
3,581,34.0,2,1,1.0,1.0,11.345227,11.91092,True,False,False,False,True
4,716,33.0,5,2,1.0,1.0,9.620384,0.00000,False,False,True,False,True


In [17]:


rf = RandomForestClassifier(random_state=1722)

scoring = {"accuracy", "precision", "recall", "f1", "roc_auc"}

param_dist = {
    'n_estimators': [500],
    'max_depth': [10],
    'min_samples_split': [5],
    'min_samples_leaf': [2],
    'max_features': ['log2']
}
#param_dist = {
    #'n_estimators': [100, 200, 300, 500],
    #'max_depth': [None, 10, 20, 30],
    #'min_samples_split': [2, 5, 10],
    #'min_samples_leaf': [1, 2, 4],
    #'max_features': ['sqrt', 'log2']
#}

random_search = RandomizedSearchCV(
    rf,
    param_dist,
    n_iter=1,
    cv=5,
    n_jobs=-1,
    scoring=scoring,
    refit="roc_auc"
)



In [18]:
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1722),
                   n_iter=1, n_jobs=-1,
                   param_distributions={'max_depth': [10],
                                        'max_features': ['log2'],
                                        'min_samples_leaf': [2],
                                        'min_samples_split': [5],
                                        'n_estimators': [500]},
                   refit='roc_auc',
                   scoring={'accuracy', 'recall', 'f1', 'precision', 'roc_auc'})

In [19]:
random_search.best_params_
#{'n_estimators': 500,
# 'min_samples_split': 5,
# 'min_samples_leaf': 2,
# 'max_features': 'log2',
# 'max_depth': 10}

{'n_estimators': 500,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'log2',
 'max_depth': 10}

In [20]:
random_search.best_score_
#0.8872367824663965

0.8872367824663965

In [21]:
def make_results(model_name: str, model_object, metric: str):
    """
    Arguments:
        model_name (string): what you want the model to be called in the output table
        model_object: a fit GridSearchCV object
        metric (string): precision, recall, f1, accuracy, or roc_auc
    
    Returns a pandas df with the f1, recall, precision, accuracy, and roc_auc scores
    for the model with the best mean 'metric' score across all validation folds.
    """
    
    # Create dictionary that maps input metric to actual metric name in GridSearchCV
    metric_dict = {
        'roc_auc': 'mean_test_roc_auc',
        'precision': 'mean_test_precision', 
        'recall': 'mean_test_recall',
        'f1': 'mean_test_f1',
        'accuracy': 'mean_test_accuracy'
    }
    
    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)
    
    # Isolate the row of the df with the max(metric) score
    best_estimator_results = cv_results.iloc[cv_results[metric_dict[metric]].idxmax(), :]
    
    # Extract accuracy, precision, recall, and f1 score from that row
    roc_auc = best_estimator_results.mean_test_roc_auc
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy
    
    # Create table of results
    table = pd.DataFrame({
        'model': [model_name],
        'precision': [precision],
        'recall': [recall], 
        'f1': [f1],
        'accuracy': [accuracy],
        'roc_auc': [roc_auc]
    })
    
    return table

In [22]:
rf1_cv_results = make_results("random forest v1",random_search, "roc_auc")
print(rf1_cv_results)

#              model  precision    recall        f1  accuracy   roc_auc
#0  random forest cv    0.76682  0.507288  0.610575  0.863089  0.887237

              model  precision    recall        f1  accuracy   roc_auc
0  random forest v1    0.76682  0.507288  0.610575  0.863089  0.887237


In [23]:
#version 2

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss',random_state=1722)

scoring = {"accuracy", "precision", "recall", "f1", "roc_auc"}

#param_dist = {
#    'n_estimators': [100,200,500],
#    'max_depth': [3,5,7,10],
#    'learning_rate': [0.01, 0.05, 0.1, 0.2],
#    'min_child_weight': [1, 3, 5],
#    'colsample_bytree': np.arange(0.5, 1.0, 0.1)
#}
param_dist = {
    'n_estimators': [500],
    'max_depth': [3],
    'learning_rate': [0.1],
    'min_child_weight': [3],
    'colsample_bytree': [0.6]
}

random_search1 = RandomizedSearchCV(
    xgb,
    param_dist,
    n_iter=50,
    cv=5,
    n_jobs=-1,
    scoring=scoring,
    refit="roc_auc",
    random_state=1722
)



In [24]:
%%time
random_search1.fit(X_train, y_train)

CPU times: user 9.32 s, sys: 110 ms, total: 9.43 s
Wall time: 14.5 s


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning...
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=1722, ...),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6],
                                        'learning_rate': [0.1],
                                        'max_depth': [3],
                                        'min_child_weight': [3],
                                        'n_estimators': [500]},
                   random_state=1722, refit='roc_auc',
                   scoring={'accuracy', 'recall', 'f1', 'precision', 'roc_auc'})

In [25]:
random_search1.best_params_
#{'n_estimators': 500,
#'min_child_weight': 3,
#'max_depth': 3,
#'learning_rate': 0.1,
#'colsample_bytree': 0.6}

{'n_estimators': 500,
 'min_child_weight': 3,
 'max_depth': 3,
 'learning_rate': 0.1,
 'colsample_bytree': 0.6}

In [26]:
random_search1.best_score_
#0.8898609732518489

0.8898609732518489

In [27]:
xgb_cv_results = make_results("xgb v1",random_search1, "roc_auc")
print(rf1_cv_results)
print(xgb_cv_results)
#              model  precision    recall        f1  accuracy   roc_auc
#0  random forest v1    0.76682  0.507288  0.610575  0.863089  0.887237
#    model  precision    recall        f1  accuracy   roc_auc
#0  xgb v1    0.74883  0.551158  0.634941    0.8659  0.889826

              model  precision    recall        f1  accuracy   roc_auc
0  random forest v1    0.76682  0.507288  0.610575  0.863089  0.887237
    model  precision    recall        f1  accuracy   roc_auc
0  xgb v1   0.747666  0.550643  0.634176  0.865579  0.889861


In [28]:
lgbm_v1 = LGBMClassifier(random_state=1722)

scoring = {
    "accuracy": "accuracy",
    "precision": "precision",
    "recall": "recall",
    "f1": "f1",
    "roc_auc": "roc_auc"
}

param_dist = {
    'n_estimators': [100,200,500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'num_leaves': [20, 31, 40, 50],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}


random_search2 = RandomizedSearchCV(
    lgbm_v1,
    param_dist,
    n_iter=50,
    cv=5,
    n_jobs=-1,
    scoring=scoring,
    refit="roc_auc",
    random_state=1722
)


In [29]:
%%time
random_search2.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 27937, number of negative: 104090
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044161 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 863
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304
[LightGBM] [Info] Number of positive: 27937, number of negative: 104090
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031523 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 862
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304
[LightGBM]

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(random_state=1722), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.7, 0.8, 0.9,
                                                             1.0],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'n_estimators': [100, 200, 500],
                                        'num_leaves': [20, 31, 40, 50],
                                        'reg_alpha': [0, 0.1, 0.5],
                                        'reg_lambda': [0, 0.1, 0.5]},
                   random_state=1722, refit='roc_auc',
                   scoring={'accuracy': 'accuracy', 'f1': 'f1',
                            'precision': 'precision', 'recall': 'recall',
                            'roc_auc': 'roc_auc'})

In [30]:
random_search2.best_params_
#{'reg_lambda': 0.1,
#'reg_alpha': 0.5,
#'num_leaves': 31,
#'n_estimators': 500,
#'learning_rate': 0.05,
#'colsample_bytree': 0.7}

{'reg_lambda': 0.1,
 'reg_alpha': 0.5,
 'num_leaves': 31,
 'n_estimators': 500,
 'learning_rate': 0.05,
 'colsample_bytree': 0.7}

In [31]:
random_search2.best_score_
#np.float64(0.8900515382076835)

0.8900515382076835

In [32]:
lgbm_cv_results = make_results("lgbm v1",random_search1, "roc_auc")
print(rf1_cv_results)
print(xgb_cv_results)
print(lgbm_cv_results)
#              model  precision    recall        f1  accuracy   roc_auc
#0  random forest v1    0.76682  0.507288  0.610575  0.863089  0.887237
#    model  precision    recall        f1  accuracy   roc_auc
#0  xgb v1    0.74883  0.551158  0.634941    0.8659  0.889826
#     model  precision    recall        f1  accuracy   roc_auc
#0  lgbm v1    0.74883  0.551158  0.634941    0.8659  0.889826

              model  precision    recall        f1  accuracy   roc_auc
0  random forest v1    0.76682  0.507288  0.610575  0.863089  0.887237
    model  precision    recall        f1  accuracy   roc_auc
0  xgb v1   0.747666  0.550643  0.634176  0.865579  0.889861
     model  precision    recall        f1  accuracy   roc_auc
0  lgbm v1   0.747666  0.550643  0.634176  0.865579  0.889861


In [33]:
# Ver las top 10 features más importantes
importances = pd.Series(
    random_search.best_estimator_.feature_importances_, 
    index=X_train.columns
).sort_values(ascending=False)

print(importances.head(10))

Age                   0.356367
NumOfProducts         0.310523
IsActiveMember        0.104806
LogBalance            0.059748
Geography_Germany     0.057782
Gender_Female         0.022183
Gender_Male           0.021779
LogEstimatedSalary    0.018481
CreditScore           0.018071
Geography_France      0.013246
dtype: float64


In [36]:
# Ver las top 10 features más importantes
importances1 = pd.Series(
    random_search1.best_estimator_.feature_importances_, 
    index=X_train.columns
).sort_values(ascending=False)

print(importances1.head(10))

NumOfProducts        0.308581
IsActiveMember       0.246450
Age                  0.121184
Gender_Male          0.097222
Geography_Germany    0.091224
Gender_Female        0.055487
Geography_France     0.033770
LogBalance           0.022274
HasCrCard            0.007796
Geography_Spain      0.006779
dtype: float32


In [37]:
# Ver las top 10 features más importantes
importances2 = pd.Series(
    random_search2.best_estimator_.feature_importances_, 
    index=X_train.columns
).sort_values(ascending=False)

print(importances2.head(10))

CreditScore           3023
LogEstimatedSalary    2855
LogBalance            2822
Age                   2544
Tenure                1219
NumOfProducts          614
IsActiveMember         411
Gender_Female          373
Geography_Germany      332
HasCrCard              300
dtype: int32


In [38]:
comparison_df = pd.DataFrame({
    'feature': X_train.columns,
    'rf_importance': importances / importances.sum(),
    'xgb_importance': importances1 / importances1.sum(), 
    'lgbm_importance': importances2 / importances2.sum()
})
comparison_df

,feature,rf_importance,xgb_importance,lgbm_importance
Age,CreditScore,0.356367,0.121184,0.169600
CreditScore,Age,0.018071,0.003456,0.201533
Gender_Female,Tenure,0.022183,0.055487,0.024867
Gender_Male,NumOfProducts,0.021779,0.097222,0.009600
Geography_France,HasCrCard,0.013246,0.033770,0.013533
Geography_Germany,IsActiveMember,0.057782,0.091224,0.022133
Geography_Spain,LogEstimatedSalary,0.005662,0.006779,0.010667
HasCrCard,LogBalance,0.002982,0.007796,0.020000
IsActiveMember,Geography_France,0.104806,0.246450,0.027400
LogBalance,Geography_Germany,0.059748,0.022274,0.188133



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030493 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 863
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304

[LightGBM] [Info] Start training from score -1.315304

[LightGBM] [Info] Total Bins 864
[LightGBM] [Info] Number of data points in the train set: 132027, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211601 -> initscore=-1.315304
[LightGBM] [Info] Start training from score -1.315304
[LightGBM] [Info] Number of positive: 27937, number of negative: 104091
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033287 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total B

In [ ]:
#el modelo en ganar la competicion hizo 0.90585 https://www.kaggle.com/competitions/playground-series-s4e1/leaderboard
#vs                                  rf 0.887237
#vs                                 xgb 0.889826
#vs                                lgbm 0.889826
#Para continuar puedo 
# 1 priorizar hacer feature engineering con las variables mas relevantes
# 2 poner en orden las variables de mas importantes a menos importantes
# 3 hacer un modelo de ensemble con los 3 / probar mas hyperparametros / probar catboost u otros modelos NB o logistic
# 4 hacer una red neuronal no muy profunda
# Al ser uno de los primeros modeloes en hacer solo , prefiero continuar viendo otras competiciones y ganar mas experiencia ,
# que estar mucho tiempo con este modelo
